# Chatbot Agent Testing Notebook

This notebook provides a user-friendly interface for testing the Chatbot Agent with pre-generated questions.
It's designed for users with little programming experience.

## Features:
- Load and edit agent configuration settings
- Load test questions from JSON files
- Process questions through the Chatbot Agent
- Review and analyze results
- Export results with timestamps

## Getting Started:
1. Run cells in order from top to bottom
2. Edit configuration values as needed
3. Load test questions from file (generated using question_generator.ipynb)
4. Review questions before processing
5. Run the agent and review results

## Question Generation:
Use the separate `question_generator.ipynb` notebook to create test questions, then load them here.

In [ ]:
# Import required libraries
import json
import os
import sys
from datetime import datetime
from pathlib import Path

import ipywidgets as widgets
import pandas as pd
from ruamel.yaml import YAML  # Changed from: import yaml
from IPython.display import display, HTML, clear_output

# Set the working directory to the root of the project
os.chdir('/workspace')

# Add workspace to path for imports (this helps with relative imports)
sys.path.insert(0, '/workspace')

# Import our system components
from src.nodes.chatbot_agent import ChatbotAgentNode
from src.core.config.agent_config_manager import AgentConfigManager
from src.integrations.llm_providers import LLMProviderFactory

print("✅ All libraries imported successfully!")
print("Ready to start testing the Chatbot Agent.")

## Step 1: Load Configuration Settings

The following cell loads the current configuration for the Chatbot Agent.
You can edit these values to customize the agent's behavior.

In [ ]:
# Load configuration from files and create temporary editable copies
config_base_path = Path('/workspace/config')
agent_config_path = config_base_path / 'agents' / 'chatbot_agent'
temp_config_dir = Path('/tmp/chatbot_agent_configs')

def load_and_create_temp_configs():
    """Load all configuration files and create temporary editable copies with comments preserved"""
    configs = {}
    
    # Create YAML instance for comment preservation
    yaml = YAML()
    yaml.preserve_quotes = True
    yaml.default_flow_style = False
    
    # Create temp directory
    temp_config_dir.mkdir(exist_ok=True)
    
    # Load original files for parsing (to access data)
    with open(agent_config_path / 'config.yaml', 'r') as f:
        configs['agent'] = yaml.load(f)
    
    with open(agent_config_path / 'prompts.yaml', 'r') as f:
        configs['prompts'] = yaml.load(f)
    
    with open(agent_config_path / 'models.yaml', 'r') as f:
        configs['models'] = yaml.load(f)
    
    # Load shared models for reference
    with open(config_base_path / 'shared' / 'models.yaml', 'r') as f:
        configs['shared_models'] = yaml.load(f)
    
    # Create temp file paths
    temp_agent_path = temp_config_dir / 'config.yaml'
    temp_prompts_path = temp_config_dir / 'prompts.yaml'
    temp_models_path = temp_config_dir / 'models.yaml'
    
    # Copy original files to temp directory to preserve comments and formatting
    import shutil
    shutil.copy2(agent_config_path / 'config.yaml', temp_agent_path)
    shutil.copy2(agent_config_path / 'prompts.yaml', temp_prompts_path)  
    shutil.copy2(agent_config_path / 'models.yaml', temp_models_path)
    
    return configs, {
        'agent_config': temp_agent_path,
        'prompts_config': temp_prompts_path,
        'models_config': temp_models_path
    }

# Load configurations and create temp files
configs, temp_file_paths = load_and_create_temp_configs()

print("📁 Configuration files loaded and temporary copies created with comments preserved!")
print(f"Agent name: {configs['agent']['agent']['name']}")
print(f"Agent version: {configs['agent']['agent']['version']}")

# Get preferred model from models config (not agent config)
preferred_model = "Unknown"
if 'primary_model' in configs['models']:
    preferred_model = configs['models']['primary_model']
elif 'preferred' in configs['models']:
    preferred_model = configs['models']['preferred']

print(f"Preferred model: {preferred_model}")
print(f"\n💾 Temporary config files created at:")
for config_type, path in temp_file_paths.items():
    print(f"  {config_type}: {path}")
print(f"\n💡 These temp files retain original comments and can be edited directly in Step 2.")

## Step 2: Editable Configuration Settings

Edit these settings to customize how the Chatbot Agent behaves.
These variables map directly to the configuration files and can be exported later.

In [ ]:
# Display and edit configuration files in separate windows

def load_config_file_contents():
    """Load current config file contents from temp files"""
    with open(temp_file_paths['agent_config'], 'r') as f:
        agent_config_content = f.read()
    with open(temp_file_paths['prompts_config'], 'r') as f:
        prompts_config_content = f.read()
    with open(temp_file_paths['models_config'], 'r') as f:
        models_config_content = f.read()
    
    return agent_config_content, prompts_config_content, models_config_content

# Load current config file contents
agent_config_content, prompts_config_content, models_config_content = load_config_file_contents()

print("⚙️ Configuration File Editor")
print("Edit the YAML configuration files below and use the Save buttons to apply changes.")
print("Changes are saved to temporary files and will be used in Step 5.\n")

# Create text areas for each config file
print("📄 1. Agent Configuration (config.yaml)")
print("Contains: agent settings, behavior, escalation thresholds")

agent_config_editor = widgets.Textarea(
    value=agent_config_content,
    description="",
    layout=widgets.Layout(width='100%', height='250px'),
    style={'description_width': '0px'}
)

def save_agent_config(button):
    """Save agent config changes with comments preserved"""
    try:
        # Create YAML instance for validation
        yaml = YAML()
        yaml.preserve_quotes = True
        yaml.default_flow_style = False
        
        # Validate YAML syntax
        yaml.load(agent_config_editor.value)
        
        # Save to temp file (preserves comments in the editor content)
        with open(temp_file_paths['agent_config'], 'w') as f:
            f.write(agent_config_editor.value)
        
        print("✅ Agent config saved successfully with comments preserved!")
        
    except Exception as e:
        print(f"❌ YAML syntax error in agent config: {e}")

agent_save_btn = widgets.Button(description="Save Agent Config", button_style='success')
agent_save_btn.on_click(save_agent_config)

display(agent_config_editor)
display(agent_save_btn)

print("\n📄 2. Prompts Configuration (prompts.yaml)")
print("Contains: system prompts, response guidelines, communication style")

prompts_config_editor = widgets.Textarea(
    value=prompts_config_content,
    description="",
    layout=widgets.Layout(width='100%', height='250px'),
    style={'description_width': '0px'}
)

def save_prompts_config(button):
    """Save prompts config changes with comments preserved"""
    try:
        # Create YAML instance for validation
        yaml = YAML()
        yaml.preserve_quotes = True
        yaml.default_flow_style = False
        
        # Validate YAML syntax
        yaml.load(prompts_config_editor.value)
        
        # Save to temp file (preserves comments in the editor content)
        with open(temp_file_paths['prompts_config'], 'w') as f:
            f.write(prompts_config_editor.value)
        
        print("✅ Prompts config saved successfully with comments preserved!")
        
    except Exception as e:
        print(f"❌ YAML syntax error in prompts config: {e}")

prompts_save_btn = widgets.Button(description="Save Prompts Config", button_style='success')
prompts_save_btn.on_click(save_prompts_config)

display(prompts_config_editor)
display(prompts_save_btn)

print("\n📄 3. Models Configuration (models.yaml)")
print("Contains: preferred model, fallback models, model-specific settings")

# Show available model aliases from shared models config
def display_available_models():
    """Display available model aliases and their actual models"""
    try:
        shared_models = configs['shared_models']
        
        # Extract model aliases and models sections
        model_aliases = shared_models.get('model_aliases', {})
        models = shared_models.get('models', {})
        
        if not model_aliases:
            print("❌ No model aliases found in shared configuration")
            return
            
        print("\n🔍 Available Model Aliases:")
        print("Use these aliases in your models configuration below:\n")
        
        # Group by provider for better organization
        providers = {}
        for alias, actual_model_name in model_aliases.items():
            # Get model details from models section
            model_details = models.get(actual_model_name, {})
            provider = model_details.get('type', 'unknown')
            description = model_details.get('description', '')
            
            if provider not in providers:
                providers[provider] = []
            providers[provider].append({
                'alias': alias,
                'model_name': actual_model_name,
                'description': description
            })
        
        # Display by provider
        for provider, provider_models in providers.items():
            print(f"📡 {provider.upper()} Provider:")
            for model in provider_models:
                desc = f" - {model['description']}" if model['description'] else ""
                print(f"  • {model['alias']} → {model['model_name']}{desc}")
            print()
        
        # Show current configuration
        try:
            # Use ruamel.yaml for parsing
            yaml = YAML()
            yaml.preserve_quotes = True
            current_models_config = yaml.load(models_config_content)
            current_preferred = current_models_config.get('primary_model', current_models_config.get('preferred', 'unknown'))
            current_fallback = current_models_config.get('fallback', [])
        except:
            current_preferred = 'unknown'
            current_fallback = []
        
        print(f"📋 Current Models Configuration:")
        print(f"  Preferred: {current_preferred}")
        print(f"  Fallback: {current_fallback}")
        print()
        
        print("💡 Tips for editing:")
        print("  • Change 'primary_model' or 'preferred' to any alias from the list above")
        print("  • Add/remove aliases in the 'fallback' list")
        print("  • Aliases are case-sensitive")
        print("  • Invalid aliases will cause errors during processing")
        print()
        
    except Exception as e:
        print(f"❌ Error loading available models: {e}")
        print("Continuing with models configuration editor...\n")

# Display available models before showing the editor
display_available_models()

models_config_editor = widgets.Textarea(
    value=models_config_content,
    description="",
    layout=widgets.Layout(width='100%', height='200px'),
    style={'description_width': '0px'}
)

def save_models_config(button):
    """Save models config changes with comments preserved"""
    try:
        # Create YAML instance for validation
        yaml = YAML()
        yaml.preserve_quotes = True
        yaml.default_flow_style = False
        
        # Validate YAML syntax
        parsed_config = yaml.load(models_config_editor.value)
        
        # Additional validation for model aliases
        if isinstance(parsed_config, dict):
            preferred = parsed_config.get('primary_model') or parsed_config.get('preferred')
            fallback = parsed_config.get('fallback', [])
            
            # Get available aliases
            model_aliases = configs['shared_models'].get('model_aliases', {})
            
            # Check if preferred model exists
            if preferred and preferred not in model_aliases:
                print(f"⚠️ Warning: Preferred model '{preferred}' not found in available model aliases")
            
            # Check fallback models
            for fb_model in fallback:
                if fb_model not in model_aliases:
                    print(f"⚠️ Warning: Fallback model '{fb_model}' not found in available model aliases")
        
        # Save to temp file (preserves comments in the editor content)
        with open(temp_file_paths['models_config'], 'w') as f:
            f.write(models_config_editor.value)
        
        print("✅ Models config saved successfully with comments preserved!")
        
    except Exception as e:
        print(f"❌ YAML syntax error in models config: {e}")

models_save_btn = widgets.Button(description="Save Models Config", button_style='success')
models_save_btn.on_click(save_models_config)

display(models_config_editor)
display(models_save_btn)

# Save All button for convenience
def save_all_configs(button):
    """Save all config changes at once"""
    save_agent_config(None)
    save_prompts_config(None)
    save_models_config(None)

print("\n💾 Save All Changes")
save_all_btn = widgets.Button(description="Save All Configs", button_style='info')
save_all_btn.on_click(save_all_configs)
display(save_all_btn)

print(f"\n💾 Temp config files location:")
for config_type, path in temp_file_paths.items():
    print(f"  {config_type}: {path}")

print("\n💡 Tips:")
print("  • Edit YAML directly in the text areas above")
print("  • Use Save buttons to apply changes to temp files")
print("  • YAML syntax is validated before saving")
print("  • Model aliases are validated against available models")
print("  • Changes will be used in Step 5 when processing questions")
print("  • Original config files remain unchanged")
print("  • Comments and formatting are preserved during editing!")

## Step 2.5: Understanding Chatbot Internal Logic

Now that you've seen the configuration settings, let's understand how the chatbot processes customer queries and how your configuration changes affect its behavior.

### How the Chatbot Agent Works Internally

The Chatbot Agent follows a sophisticated multi-stage process for each customer interaction:

#### 1. **Initial Assessment Phase**
- **Query Analysis**: The chatbot first analyzes the customer's question to understand:
  - Intent and urgency level
  - Technical complexity 
  - Emotional tone (frustration, confusion, etc.)
  - Required information categories
- **Context Integration**: Retrieves previous conversation history and relevant customer data
- **Confidence Scoring**: Assigns a confidence score (0.0-1.0) based on how well it understands the query

#### 2. **Response Generation Phase**
- **System Prompt Application**: Uses the `system` prompt from `prompts.yaml` to establish personality and behavior
- **Model Selection**: Chooses the appropriate AI model based on query complexity:
  - Simple queries → Uses `primary_model` from `models.yaml`
  - Complex technical queries → May escalate to premium models if configured
  - Reasoning-heavy queries → Uses models optimized for reasoning
- **Response Crafting**: Generates response following the guidelines in `prompts.yaml`

#### 3. **Quality Control Phase**
- **Escalation Threshold Check**: Compares confidence score against thresholds in `config.yaml`:
  - If confidence < `escalation.confidence_threshold` → Triggers escalation
  - If query complexity > agent capabilities → Routes to human specialist
- **Response Validation**: Ensures response meets quality standards
- **Safety Checks**: Verifies no inappropriate or harmful content

### How Configuration Settings Affect Behavior

#### **Temperature Setting** (`config.yaml: settings.temperature`)
- **Low (0.0-0.3)**: More deterministic, consistent responses
  - *Best for*: Policy explanations, factual information
  - *Trade-off*: May sound robotic or repetitive
- **Medium (0.4-0.7)**: Balanced creativity and consistency  
  - *Best for*: General customer service, explanations
  - *Trade-off*: Good balance of personality and accuracy
- **High (0.8-1.0)**: More creative, varied responses
  - *Best for*: Empathetic responses, complex problem-solving
  - *Trade-off*: May be less predictable or occasionally off-topic

#### **System Prompt** (`prompts.yaml: system`)
This is the chatbot's "personality and instructions." Key components:
- **Role Definition**: "You are a professional customer service chatbot..."
- **Communication Style**: Friendly, helpful, professional tone guidelines
- **Response Structure**: How to organize information in responses
- **Escalation Guidance**: When and how to refer customers to human agents
- **Service Standards**: Quality expectations and customer satisfaction goals

*Example Impact*: Adding "Be concise and direct" to the system prompt will make responses shorter and more to-the-point.

#### **Model Selection** (`models.yaml`)
- **Primary Model**: The default model used for most queries
  - `anthropic_general_standard`: Balanced performance, good for most queries
  - `local_general_standard`: Faster local processing, may be less sophisticated
  - `openai_general_standard`: High-quality responses, requires API access
- **Fallback Models**: Used when primary model is unavailable
- **Model Preferences by Query Type**: Different models for different scenarios
  - `general_queries.primary`: Standard customer service questions
  - `complex_queries.primary`: Technical or complicated issues  
  - `escalation_queries.primary`: When preparing to transfer to human

#### **Escalation Thresholds** (`config.yaml: escalation`)
- **Confidence Threshold**: Minimum confidence to attempt answering
  - Lower values → More willing to try answering difficult questions
  - Higher values → More conservative, escalates uncertain queries faster
- **Max Attempts**: How many clarification questions to ask before escalating
- **Priority Routing**: Which human departments to route different query types

#### **Behavior Settings** (`config.yaml: behavior`)
- **Response Length Preferences**: Target length for different response types
- **Follow-up Strategy**: How proactive to be in asking clarifying questions
- **Empathy Level**: How much emotional intelligence to apply
- **Technical Depth**: How detailed to get with technical explanations

### Real-World Impact Examples

#### Scenario: Customer asks "My claim was denied, what do I do?"

**With Conservative Settings** (high escalation threshold, low temperature):
- High confidence threshold → May immediately escalate to human agent
- Low temperature → Gives standard, policy-based response
- Result: Quick escalation, consistent but potentially impersonal

**With Balanced Settings** (medium thresholds, medium temperature):
- Medium confidence → Asks clarifying questions first
- Balanced temperature → Provides empathetic but accurate response
- Result: Attempts to help while showing understanding

**With Aggressive Settings** (low escalation threshold, high temperature):
- Low confidence threshold → Attempts detailed explanation
- High temperature → Creative, personalized response approach
- Result: More comprehensive help but potentially longer interaction

### Configuration Optimization Tips

1. **For High-Volume, Simple Queries**:
   - Use faster models (`local_general_standard`)
   - Lower temperature for consistency
   - Higher escalation thresholds to reduce human load

2. **For Complex Technical Support**:
   - Use reasoning-optimized models (`anthropic_reasoning_premium`)
   - Medium-high temperature for creative problem-solving
   - Lower escalation thresholds to ensure accuracy

3. **For Frustrated Customers**:
   - Emphasize empathy in system prompts
   - Medium temperature for personalized responses
   - Faster escalation to human agents when emotions are high

4. **For Cost Optimization**:
   - Prefer local models over API-based models
   - Use budget models for simple queries
   - Set appropriate escalation thresholds to balance cost and quality

### Monitoring and Adjustment

The chatbot's performance can be monitored through:
- **Confidence Scores**: Track how certain the chatbot is about responses
- **Escalation Rates**: Monitor how often queries are passed to humans
- **Customer Satisfaction**: Measure response quality and helpfulness
- **Response Times**: Balance quality with speed requirements
- **Conversation Length**: Optimize for efficiency while maintaining quality

*Understanding these internals helps you make informed configuration changes that improve customer experience while managing operational costs.*

## Step 3: Load Test Questions

Load test questions from a JSON file. Use question_generator.ipynb to create new questions.

In [ ]:
# File upload widget for loading questions
file_upload = widgets.FileUpload(
    accept='.json',
    multiple=False,
    description='Upload questions file:'
)

# Instructions for file format
print("📝 Load Test Questions")
print("\n💡 How to get test questions:")
print("1. Use question_generator.ipynb to create test questions")
print("2. Upload the generated JSON file below")
print("3. Supported formats: Full export or questions-only JSON")
print("\n📄 Expected JSON format:")
print("- Full export: {'metadata': {...}, 'questions': [...]}")
print("- Questions only: [{'id': 1, 'question': '...', 'customer_type': '...', 'complexity': '...'}]")
print("\n📁 Upload your questions file:")
display(file_upload)

In [ ]:
# Load test questions from uploaded file
test_questions = []

def load_questions_from_file(file_content, filename):
    """Load questions from uploaded JSON file"""
    try:
        # Handle different content types
        if isinstance(file_content, memoryview):
            # Convert memoryview to bytes
            content_bytes = file_content.tobytes()
        elif hasattr(file_content, 'decode'):
            # Already bytes
            content_bytes = file_content
        else:
            # Convert to bytes if it's a string or other type
            content_bytes = str(file_content).encode('utf-8')
        
        # Decode to string and parse JSON
        data = json.loads(content_bytes.decode('utf-8'))
        
        # Handle different JSON formats
        if isinstance(data, dict):
            # Full export format with metadata
            if 'questions' in data:
                questions = data['questions']
                metadata = data.get('metadata', {})
                print(f"📄 Loaded file with metadata:")
                print(f"  Generation model: {metadata.get('generation_model', 'unknown')}")
                print(f"  Generation timestamp: {metadata.get('generation_timestamp', 'unknown')}")
                print(f"  Question count: {metadata.get('question_count', len(questions))}")
                return questions
            else:
                # Single question object
                return [data]
        elif isinstance(data, list):
            # Questions-only format
            return data
        else:
            print(f"❌ Unexpected data format: {type(data)}")
            return []
            
    except json.JSONDecodeError as e:
        print(f"❌ JSON parsing error: {e}")
        return []
    except Exception as e:
        print(f"❌ Error loading file: {e}")
        print(f"   File content type: {type(file_content)}")
        return []

def validate_questions(questions):
    """Validate and normalize question format"""
    validated_questions = []
    
    for i, q in enumerate(questions):
        if isinstance(q, dict):
            # Ensure required fields exist
            validated_q = {
                "id": q.get("id", i + 1),
                "question": q.get("question", f"Question {i + 1}"),
                "customer_type": q.get("customer_type", "normal"),
                "complexity": q.get("complexity", "medium")
            }
            validated_questions.append(validated_q)
        else:
            # Convert string to dict if needed
            validated_q = {
                "id": i + 1,
                "question": str(q),
                "customer_type": "normal",
                "complexity": "medium"
            }
            validated_questions.append(validated_q)
    
    return validated_questions

# Process uploaded file
if file_upload.value:
    uploaded_file = None
    filename = None
    file_content = None
    
    # Handle different file upload widget formats
    if isinstance(file_upload.value, tuple) and len(file_upload.value) > 0:
        print("📋 Using tuple format")
        uploaded_file = file_upload.value[0]
        filename = uploaded_file['name']
        file_content = uploaded_file['content']
        print(f"🔍 File content type: {type(file_content)}")
    elif isinstance(file_upload.value, dict) and len(file_upload.value) > 0:
        print("📋 Using dict format")
        uploaded_file = list(file_upload.value.values())[0]
        filename = uploaded_file['metadata']['name']
        file_content = uploaded_file['content']
        print(f"🔍 File content type: {type(file_content)}")
    else:
        print(f"❌ Unable to read uploaded file format")
        print(f"   Type: {type(file_upload.value)}")
        print(f"   Length: {len(file_upload.value) if hasattr(file_upload.value, '__len__') else 'No length'}")
        print(f"   Content: {file_upload.value}")
    
    if uploaded_file and filename and file_content is not None:
        print(f"📁 Loading questions from: {filename}")
        
        raw_questions = load_questions_from_file(file_content, filename)
        
        if raw_questions:
            test_questions = validate_questions(raw_questions)
            print(f"✅ Loaded {len(test_questions)} test questions")
            
            # Display first few questions as preview
            print("\n📋 Preview of loaded questions:")
            for i, q in enumerate(test_questions[:3]):
                print(f"  {i+1}. {q['question']} [{q['customer_type']}]")
            if len(test_questions) > 3:
                print(f"  ... and {len(test_questions) - 3} more questions")
                
            # Show distribution
            df_preview = pd.DataFrame(test_questions)
            print("\n📊 Question Distribution:")
            print(f"  Customer types: {dict(df_preview['customer_type'].value_counts())}")
            print(f"  Complexities: {dict(df_preview['complexity'].value_counts())}")
            
        else:
            print("❌ No questions loaded from file")
    else:
        print("❌ Error accessing uploaded file")
        print(f"   uploaded_file: {uploaded_file is not None}")
        print(f"   filename: {filename}")
        print(f"   file_content: {file_content is not None}")
else:
    print("⚠️ Please upload a JSON file with test questions.")
    print("💡 Use question_generator.ipynb to create test questions first.")

## Step 4: Review and Edit Questions

Review the loaded questions and make any edits before processing.

In [ ]:
# Create interactive question editor
if test_questions:
    print("📝 Question Editor - You can modify questions before processing")
    print("Edit the questions in the table below, then run the next cell to process them.\n")
    
    # Convert to DataFrame for easy editing
    df = pd.DataFrame(test_questions)
    
    # Ensure required columns exist
    if 'id' not in df.columns:
        df['id'] = range(1, len(df) + 1)
    if 'customer_type' not in df.columns:
        df['customer_type'] = 'normal'
    if 'complexity' not in df.columns:
        df['complexity'] = 'medium'
    
    # Display editable table
    print("Current questions (you can edit the JSON below if needed):")
    display(df)
    
    # Show JSON for manual editing if needed
    questions_json = widgets.Textarea(
        value=json.dumps(test_questions, indent=2),
        description="Questions JSON:",
        layout=widgets.Layout(width='100%', height='200px')
    )
    
    print("\nAdvanced: Edit questions as JSON (optional):")
    display(questions_json)
    
    def update_questions_from_json():
        """Update questions from the JSON editor"""
        global test_questions
        try:
            test_questions = json.loads(questions_json.value)
            print("✅ Questions updated from JSON editor")
        except Exception as e:
            print(f"❌ Error parsing JSON: {e}")
    
    # Button to update from JSON
    update_btn = widgets.Button(description="Update from JSON")
    update_btn.on_click(lambda b: update_questions_from_json())
    display(update_btn)
    
else:
    print("⚠️ No questions loaded. Please upload a questions file in the previous step.")

In [ ]:
## Step 6: Complete Conversations with Customer AI

Continue conversations between customer AI and chatbot AI until natural resolution or escalation.

## Step 7: Export Results and Settings

Save the results and configuration settings to files with timestamps.

In [ ]:
## Step 8: Review Results

Display and analyze the final results.

In [ ]:
# Review and analyze conversation results
if 'results' in locals() and results:
    print("📋 Conversation Results Review and Analysis\n")
    
    # Create DataFrame for analysis
    df_results = pd.DataFrame([
        {
            'id': r.get('id'),
            'customer_type': r.get('customer_type'),
            'complexity': r.get('complexity'),
            'total_turns': r.get('total_turns', 0),
            'final_outcome': r.get('final_outcome'),
            'customer_satisfaction': r.get('customer_satisfaction', 0),
            'conversation_complete': r.get('conversation_complete', False)
        }
        for r in results
    ])
    
    print("=== CONVERSATION SUMMARY STATISTICS ===")
    print(f"Total conversations: {len(df_results)}")
    print(f"Completed conversations: {df_results['conversation_complete'].sum()}")
    print(f"Average turns per conversation: {df_results['total_turns'].mean():.1f}")
    print(f"Average customer satisfaction: {df_results['customer_satisfaction'].mean():.3f}")
    
    # Outcome distribution
    print("\n=== CONVERSATION OUTCOMES ===")
    outcome_counts = df_results['final_outcome'].value_counts()
    for outcome, count in outcome_counts.items():
        percentage = count / len(df_results) * 100
        print(f"{outcome}: {count} ({percentage:.1f}%)")
    
    # Analysis by customer type
    print("\n=== ANALYSIS BY CUSTOMER TYPE ===")
    customer_analysis = df_results.groupby('customer_type').agg({
        'total_turns': 'mean',
        'customer_satisfaction': 'mean',
        'conversation_complete': 'sum'
    }).round(3)
    customer_analysis.columns = ['Avg Turns', 'Avg Satisfaction', 'Completed']
    print(customer_analysis)
    
    # Satisfaction rate by customer type
    print("\nSatisfaction Rate by Customer Type:")
    for ctype in df_results['customer_type'].unique():
        ctype_data = df_results[df_results['customer_type'] == ctype]
        satisfied = len(ctype_data[ctype_data['final_outcome'] == 'satisfied'])
        total = len(ctype_data)
        print(f"  {ctype}: {satisfied}/{total} ({satisfied/total*100:.1f}%)")
    
    # Analysis by complexity
    print("\n=== ANALYSIS BY COMPLEXITY ===")
    complexity_analysis = df_results.groupby('complexity').agg({
        'total_turns': 'mean',
        'customer_satisfaction': 'mean',
        'conversation_complete': 'sum'
    }).round(3)
    complexity_analysis.columns = ['Avg Turns', 'Avg Satisfaction', 'Completed']
    print(complexity_analysis)
    
    # Show detailed results table
    print("\n=== CONVERSATION SUMMARY TABLE ===")
    display_df = df_results[['id', 'customer_type', 'complexity', 'total_turns', 'final_outcome', 'customer_satisfaction']].copy()
    display_df['customer_satisfaction'] = display_df['customer_satisfaction'].round(3)
    display(display_df)
    
    # Show sample full conversations
    print("\n=== SAMPLE FULL CONVERSATIONS ===")
    
    # Show 2 satisfied and 2 escalated conversations for analysis
    satisfied_conversations = [r for r in results if r.get('final_outcome') == 'satisfied']
    escalated_conversations = [r for r in results if 'escalation' in r.get('final_outcome', '')]
    
    def display_conversation(conversation_data, max_turns=3):
        """Display a conversation with turn-by-turn analysis"""
        conv_id = conversation_data.get('id')
        customer_type = conversation_data.get('customer_type')
        complexity = conversation_data.get('complexity')
        outcome = conversation_data.get('final_outcome')
        satisfaction = conversation_data.get('customer_satisfaction', 0)
        
        print(f"\n--- Conversation {conv_id} ({customer_type}, {complexity}) ---")
        print(f"Original Question: {conversation_data.get('original_question', '')}")
        print(f"Final Outcome: {outcome} (Customer Satisfaction: {satisfaction:.2f})")
        
        conversation_history = conversation_data.get('conversation_history', [])
        turns_to_show = min(max_turns, len(conversation_history))
        
        for i, turn in enumerate(conversation_history[:turns_to_show]):
            turn_num = turn.get('turn_number', i+1)
            print(f"\n  Turn {turn_num}:")
            print(f"    Customer: {turn.get('customer_query', '')[:150]}{'...' if len(turn.get('customer_query', '')) > 150 else ''}")
            print(f"    Chatbot:  {turn.get('chatbot_response', '')[:150]}{'...' if len(turn.get('chatbot_response', '')) > 150 else ''}")
            
            if 'customer_response' in turn:
                print(f"    Customer Reply: {turn.get('customer_response', '')[:150]}{'...' if len(turn.get('customer_response', '')) > 150 else ''}")
                print(f"    Satisfaction: {turn.get('customer_satisfaction', 0):.2f}")
        
        if len(conversation_history) > turns_to_show:
            print(f"  ... and {len(conversation_history) - turns_to_show} more turns")
    
    # Show satisfied conversations
    if satisfied_conversations:
        print("\n🟢 SATISFIED CUSTOMER CONVERSATIONS:")
        for conv in satisfied_conversations[:2]:
            display_conversation(conv)
    
    # Show escalated conversations  
    if escalated_conversations:
        print("\n🔴 ESCALATED CONVERSATIONS:")
        for conv in escalated_conversations[:2]:
            display_conversation(conv)
    
    # Conversation insights
    print("\n=== CONVERSATION INSIGHTS ===")
    
    # Turn analysis
    short_conversations = len(df_results[df_results['total_turns'] <= 2])
    medium_conversations = len(df_results[(df_results['total_turns'] > 2) & (df_results['total_turns'] <= 5)])
    long_conversations = len(df_results[df_results['total_turns'] > 5])
    
    print(f"Conversation Length Distribution:")
    print(f"  Short (1-2 turns): {short_conversations} ({short_conversations/len(df_results)*100:.1f}%)")
    print(f"  Medium (3-5 turns): {medium_conversations} ({medium_conversations/len(df_results)*100:.1f}%)")
    print(f"  Long (6+ turns): {long_conversations} ({long_conversations/len(df_results)*100:.1f}%)")
    
    # Satisfaction insights
    high_satisfaction = len(df_results[df_results['customer_satisfaction'] >= 0.8])
    medium_satisfaction = len(df_results[(df_results['customer_satisfaction'] >= 0.5) & (df_results['customer_satisfaction'] < 0.8)])
    low_satisfaction = len(df_results[df_results['customer_satisfaction'] < 0.5])
    
    print(f"\nCustomer Satisfaction Distribution:")
    print(f"  High (0.8+): {high_satisfaction} ({high_satisfaction/len(df_results)*100:.1f}%)")
    print(f"  Medium (0.5-0.8): {medium_satisfaction} ({medium_satisfaction/len(df_results)*100:.1f}%)")
    print(f"  Low (<0.5): {low_satisfaction} ({low_satisfaction/len(df_results)*100:.1f}%)")
    
    print("\n" + "="*80)
    print("✅ Conversation analysis complete! Check the exported files for full conversation details.")
    print("💡 Each conversation includes turn-by-turn customer-AI interaction data for detailed analysis.")
    
else:
    print("⚠️ No conversation results to review. Please complete conversations first.")